# **Classification using Neural Networks**
---
# Importing Libraries and defining useful classes and functions :

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(0)

def sigmoid(z):                         # Sigmoid Activation function
    return 1/(1+np.exp(-z))

def sigmoid_derivative (x):             # Deriviative of sigmoid function
    return sigmoid(x)*(1-sigmoid(x))

def softmax(z):                         # Softmax Activation function
    return np.exp(z)/np.sum(np.exp(z),axis = 1,keepdims = True)

def softmax_derivative(x):              # Derivative of softmax function
    return softmax(x)*(1 - softmax(x))

def Accuracy(y_pred , y_test):          # Finding accuracy by direct comparision of predicted label and test label 
    
    count = 0
    y_pred = np.argmax(y_pred,axis = 1)
    y_test = np.argmax(y_test,axis = 1)
    for i in range(y_pred.size):
        if (y_pred[i] - y_test[i] == 0):
            count+=1
    return count/y_test.size

class Layer():                                                     # Defining layers of neural network with attributes :
    def __init__(self,n_inputs,n_neurons):                         # weights , bias , outputs and error for each layer                    
        self.weights = np.random.randn(n_inputs,n_neurons)
        self.bias = np.random.randn(1,n_neurons)
    def forward(self,inputs):
        self.outputs = inputs @ self.weights + self.bias
        
    def backward(self,error,weights):
        self.error = ( error @ np.transpose(weights) ) * sigmoid_derivative(self.outputs)

    def set_error(self,error):
        self.error = error
 
def forward_propagate(x,layers,act):                             # Forward Propagation through the network to calculate outputs of each layer
                                                                 # and to store activated outputs 
    n = len(layers)-1
    for i in range(n):
                                                                 
        if i == 0 :                                              # Hidden layers 
            layers[0].forward(x)                                 # with sigmoid activation for non-linearity
            act[0] = sigmoid(layers[0].outputs)                  #
        else:                                                    #
            layers[i].forward(act[i-1])                          #
            act[i] = sigmoid(layers[i].outputs)                  #

    layers[n].forward(act[n-1])                                  # Output layer 
    act[n] = softmax(layers[n].outputs)                          # softmax activation for conversion to probabilty distribution

    return layers,act

def backward_propagate(x,y,learning_rate,layers,act):            # Backward Propagation through the network to update weights and bias
    size = act[-1].shape[0]                                      # and set error for each layer
    n = len(layers) -1
    error = y-(act[-1])                                          # error in final output

    for i in range(n,0,-1):
        if i == (n):
            error *= softmax_derivative(layers[-1].outputs)      
            layers[i].set_error(error)                           # sets error for output layer
            Jw = np.array(act[i-1]).T @ layers[i].error          # derivative of loss w.r.t weights of output layer
            Jb = np.sum(error,axis = 0)                          # derivative of loss w.r.t bias of output layer

        else :
            layers[i].backward(layers[i+1].error,layers[i+1].weights)    # sets error of hidden layer
            Jw = np.array(act[i-1]).T @ layers[i].error                  # derivative of loss w.r.t weights of ith hidden layer
            Jb = np.sum(layers[i].error,axis = 0 )                       # derivative of loss w.r.t bias of ith hidden layer

      # Updating weights and bias :

        layers[i].weights += learning_rate/size*Jw                       
        layers[i].bias += learning_rate/size*Jb                          

    layers[0].backward(layers[1].error,layers[1].weights)        # sets error of hidden layer
    Jw = np.array(x).T @ layers[0].error                         # derivative of loss w.r.t weights of first hidden layer
    Jb = np.sum(layers[0].error,axis = 0)                        # derivative of loss w.r.t bias of first hidden layer

  # Updating weights and bias of first hidden layer :

    layers[0].weights += learning_rate/size*Jw
    layers[0].bias += learning_rate/size*Jb

    return layers

# Analysis of Given Labelled data

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

df_train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/classification_train.csv",index_col = 0)
labels = df_train.label.values

itr = 0
for i in range(30000):                                     # Plotting 10 grayscale images denoting labels 0 to 9 
    if(labels[i]==itr):
        if(itr>9):
            break
        itr+=1
        xi = df_train.iloc[i,1:].to_numpy()                # from 784 = 28*28 pixels values
        xi = xi.reshape([28,28])                           # using OpenCV
        img = np.uint8(xi)
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        cv2_imshow(img)

# Checking the different labels and analysing bias in data :

classes = pd.DataFrame([np.unique(df_train.label.values),np.bincount(df_train.label.values)],index = ['labels','Number of labels'])
print(classes)                                

                     0     1     2     3     4     5     6     7     8     9
labels               0     1     2     3     4     5     6     7     8     9
Number of labels  3000  3000  3000  3000  3000  3000  3000  3000  3000  3000


# Initializing Variables and the Neural Network :

In [ ]:
# Splitting labelled data in training and testing sets :

train_data = df_train.iloc[5000:,1:].values          
test_data = df_train.iloc[:5000,1:].values
label_train = df_train.iloc[5000:,0].to_numpy()
label_test = df_train.iloc[:5000,0].to_numpy()

# Normalizating Data :

x = (train_data)/255
x_test = (test_data)/255

row , col = x.shape

# One Hot Encoding labels :

y = np.zeros([row,10])                              # Training labels 
y_test = np.zeros([label_test.shape[0],10])         # Testing labels

for i in range(row):
    y[i , label_train[i]] = 1 
    if i < label_test.shape[0] :
        y_test[i , label_test[i]] = 1               

n = 2  #2                                           # number of hidden layers
m = 20   #20                                        # number of neurons in each hidden layer
layers , act = [1]*(n+1) , [1]*(n+1)                # Creating lists to store layers and activated outputs 

for i in range(n):
    if i == 0 :
        layers[0] = Layer(784,m)                    # first hidden layer 
        layers[0].forward(x)                        # 
        act[0] = sigmoid(layers[0].outputs)         # sigmoid activation
    else:
        layers[i] = Layer(m,m)                      # hidden layers
        layers[i].forward(act[i-1])                 # 
        act[i] = sigmoid(layers[i].outputs)         # sigmoid activation

layers[n] = Layer(m,10)                             # output layer
layers[n].forward(act[n-1])                         # 
act[n] = softmax(layers[n].outputs)                 # softmax activation

# Using Stored weights and biases : (stored in .npy files)
"""Comment out below statements if need to start with random weights and biases"""

layers[0].weights = np.load("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Class/nncw1.npy")
layers[0].bias = np.load("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Class/nncb1.npy")
layers[1].weights = np.load("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Class/nncw2.npy")
layers[1].bias = np.load("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Class/nncb2.npy")
layers[2].weights = np.load("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Class/nncw3.npy")
layers[2].bias = np.load("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Class/nncb3.npy")

# Training of Neural Network via mini-batch gradient descent :

In [ ]:
learning_rate=1                                          # Hyperparameters
n_iterations = 10

start = 0
end = x.shape[0]
step = x.shape[0]//10                                    # mini-batch
                         
itr = 0                                                  # counter for number of iterations

for i in range(n_iterations):
    for k in range(start,end,step):

        layers,act = forward_propagate(x[k:k+step,:],layers,act)                             # updating outputs and activated outputs
        layers = backward_propagate(x[k:k+step,:],y[k:k+step,:],learning_rate,layers,act)    # updating weights and biases
        layers,act = forward_propagate(x,layers,act)                                         # updating outputs and activated outputs

        loss = -np.mean(y * np.log(act[-1]) + (1-y) * np.log(1-act[-1]))        # cross-entropy loss
        itr+=1                                              

        if (itr % 10 == 0):

            print("After {} iterations : ".format(itr))
            print("loss = ",loss)

            # Calculating accuracy over labelled test data :

            layers_test , act_test = forward_propagate(x_test,layers,act)       
            score = Accuracy(np.round(act_test[-1]),y_test)
            print("Accuracy = ",score*100,"%")

            # Comparing some values :

            print("Y_test = ",np.argmax(y_test[970:1000,:],axis = 1))
            print("Y_pred = ",np.argmax(act_test[-1][970:1000,:],axis = 1))


After 10 iterations : 
loss =  0.06722733160931552
Accuracy =  80.44 %
Y_test =  [1 7 6 6 1 0 4 6 4 3 1 6 4 5 7 1 9 1 5 4 6 9 6 6 1 1 1 7 1 2]
Y_pred =  [1 7 6 4 1 0 4 6 4 3 1 6 4 5 7 1 9 1 5 4 6 5 6 6 1 1 1 7 1 2]
After 20 iterations : 
loss =  0.06722203623848287
Accuracy =  80.44 %
Y_test =  [1 7 6 6 1 0 4 6 4 3 1 6 4 5 7 1 9 1 5 4 6 9 6 6 1 1 1 7 1 2]
Y_pred =  [1 7 6 4 1 0 4 6 4 3 1 6 4 5 7 1 9 1 5 4 6 5 6 6 1 1 1 7 1 2]
After 30 iterations : 
loss =  0.06721675111973331
Accuracy =  80.46 %
Y_test =  [1 7 6 6 1 0 4 6 4 3 1 6 4 5 7 1 9 1 5 4 6 9 6 6 1 1 1 7 1 2]
Y_pred =  [1 7 6 4 1 0 4 6 4 3 1 6 4 5 7 1 9 1 5 4 6 5 6 6 1 1 1 7 1 2]
After 40 iterations : 
loss =  0.06721147624323942
Accuracy =  80.46 %
Y_test =  [1 7 6 6 1 0 4 6 4 3 1 6 4 5 7 1 9 1 5 4 6 9 6 6 1 1 1 7 1 2]
Y_pred =  [1 7 6 4 1 0 4 6 4 3 1 6 4 5 7 1 9 1 5 4 6 5 6 6 1 1 1 7 1 2]
After 50 iterations : 
loss =  0.0672062116013572
Accuracy =  80.46 %
Y_test =  [1 7 6 6 1 0 4 6 4 3 1 6 4 5 7 1 9 1 5 4 6 9 6 6 1 1 1 7 1 2

# Saving weights and biases into .npy files

In [ ]:
# np.save("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Class/nncw1.npy",layers[0].weights)
# np.save("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Class/nncb1.npy",layers[0].bias)
# np.save("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Class/nncw2.npy",layers[1].weights)
# np.save("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Class/nncb2.npy",layers[1].bias)
# np.save("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Class/nncw3.npy",layers[2].weights)
# np.save("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Class/nncb3.npy",layers[2].bias)

# Predicting labels of Unlabelled data :

In [ ]:
# Unlabelled data :

df_test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/classification_test.csv")
X_test = df_test.iloc[:,1:].values
ids = df_test.iloc[:,0].values

# Standardization w.r.t Training data :

Test_data = (X_test)/255

layers_test,act_test = forward_propagate(Test_data,layers,act)
Y_pred = np.argmax(layers_test[-1].outputs,axis = 1)              # Predictions

res = pd.DataFrame([ids,Y_pred],index = ["Ids","Prediction"]).T   # Storing results into .csv file
print(res)
res.to_csv("/content/drive/MyDrive/Colab Notebooks/ML Bootcamp data/Neural_Networks_Class/nnc_result.csv")

        Ids  Prediction
0     25672           0
1     59964           1
2     66230           2
3     50801           2
4     83307           6
...     ...         ...
9995  71009           0
9996  67472           6
9997  99226           8
9998  58519           2
9999  80458           2

[10000 rows x 2 columns]
